
# Sistema de Recomendación de Videojuegos


## Metodología

El sistema de recomendación usa el método de vecinos más cercanos (K-NN) en un enfoque llamado filtrado colaborativo. Se basa en las horas de juego acumuladas (playtime) de los usuarios en cada juego para calcular qué juegos son similares.

Para hacer esto, usamos los datos de `users_items` que nos dicen qué juegos ha jugado cada usuario y cuántas horas ha jugado. Convertimos estos datos en una tabla donde las filas son juegos (identificados por `item_id`) y las columnas son usuarios. Los números en la tabla son las horas jugadas; si un usuario no ha jugado a un juego, usamos ceros.

Para hacer el cálculo más rápido, convertimos esta tabla en una matriz dispersa. Luego, usamos el algoritmo K-NN con una métrica llamada coseno, que es buena para medir similitudes entre los juegos.


In [5]:
import pandas as pd
import pickle
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

steam_games = pd.read_pickle("../Data/steam_games.pkl")
users_items = pd.read_pickle("../Data/users_items.pkl")

In [6]:
# reducir tamaño para modelo

users_items = users_items[:100000]


In [7]:
# Guardar para la API
steam_games.to_pickle("../Data/steam_games_model.pkl")
users_items.to_pickle("../Data/users_items_model.pkl")

In [8]:

# Crear una matriz de utilidad a partir de la tabla de usuarios e items
matrix_utility = users_items.pivot(
    index='item_id', columns='user_id', values='playtime_forever').fillna(0)

In [9]:
pickle.dump(matrix_utility, open("../Data/matrix_utility.pkl", "wb"))

In [10]:
# Convertir la matriz de utilidad a una matriz sparse para mejorar la eficiencia
matrix_utility_sparse = csr_matrix(matrix_utility.values)

In [11]:
model_knn = NearestNeighbors(
    metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# Entrenar el modelo
model_knn.fit(matrix_utility_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [12]:
pickle.dump(model_knn, open("../Data/model_knn.pkl", "wb"))

In [13]:
def recomendacion_juego(item_id, steam_games_df, matrix_utility, model_knn, k=5):
    item_id_str = str(item_id)

    if item_id_str not in steam_games_df['id'].astype(str).values:
        return f"El juego con ID {item_id} no fue encontrado."

    try:
        item_idx = matrix_utility.index.get_loc(item_id_str)
        distances, indices = model_knn.kneighbors(
            matrix_utility.iloc[item_idx, :].to_numpy().reshape(1, -1), n_neighbors=k+1)

        recommended_ids = [matrix_utility.index[i]
                           for i in indices.flatten()[1:]]
        recomendaciones = []

        for rec_id in recommended_ids:
            if rec_id in steam_games_df['id'].astype(str).values:
                nombre_juego = steam_games_df[steam_games_df['id'].astype(
                    str) == rec_id]['app_name'].iloc[0]
                recomendaciones.append({'id': rec_id, 'nombre': nombre_juego})
            else:
                recomendaciones.append(
                    {'id': rec_id, 'nombre': 'Nombre de juego no encontrado'})

        return recomendaciones
    except Exception as e:
        return str(e)  # Devolver el mensaje de error


recomendacion_juego("300", steam_games, matrix_utility, model_knn, k=5)

[{'id': '30', 'nombre': 'Day Of Defeat'},
 {'id': '276770', 'nombre': 'Nombre de juego no encontrado'},
 {'id': '266150', 'nombre': 'Lost Saga Na'},
 {'id': '294810', 'nombre': 'Blazblue Continuum Shift Extend'},
 {'id': '1200', 'nombre': 'Red Orchestra Ostfront 4145'}]